In [1]:
from elasticsearch import Elasticsearch

In [37]:
# %run ./RAG_MIN_SEARCH.ipynb

In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [8]:
from tqdm.auto import tqdm
import json

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [127]:
llm_index_name = "llm-course-questions"
es_client.indices.create(index=llm_index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llm-course-questions'})

In [128]:
with open('documents-llm.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
for doc in tqdm(documents):
    es_client.index(index=llm_index_name, document=doc)

  0%|          | 0/86 [00:00<?, ?it/s]

In [10]:
query = 'How do execute a command on a Kubernetes pod?'

In [25]:
def elastic_search_rag(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [33]:
def q3_elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [34]:
print(q3_elastic_search(query))

[44.50556, 35.433445, 33.70974, 33.2635, 32.589073]


In [73]:
def q4_elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [51]:
q4_query="How do copy a file to a Docker container?"

In [53]:
print(q4_elastic_search(q4_query)[2]['question'])

How do I copy files from a different folder into docker container’s working directory?


In [122]:
def q5_build_prompt(query, search_results):
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    
    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ""
    
    for doc in search_results:
        context =  context + context_template.format(question=doc['question'], text=doc['text'])+'\n\n'

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [112]:
q5_query="How do copy a file to a Docker container?"

In [113]:
search_results = q4_elastic_search(q5_query)

In [123]:
prompt = q5_build_prompt(q5_query, search_results)

In [126]:
# prompt
len(prompt)

1446

## Question 6

In [130]:
import tiktoken

In [131]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [133]:
token=encoding.encode(prompt)

In [134]:
len(token)

320

In [ ]:
def q5_rag(query):
    search_results = q4_elastic_search(query)
    prompt = q5_build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
def rag(query):
    search_results = elastic_search_rag(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer